In [30]:
import gzip, json
import mysql.connector
import gzip, json
import mysql.connector




In [37]:
# ---- CONFIG ----
INPUT_FILE = "meta_Cell_Phones_and_Accessories.jsonl"   # path to your JSONL or JSONL.GZ
LIMIT = 2000                               # number of rows to insert
TABLE_NAME = "products"                   # existing table name
DB_CONFIG = {
    "host": "127.0.0.1",
    "port": 3306,
    "user": "root",
    "password": "",
    "database": "multivendorshop"
}


In [ ]:

# Map JSON fields -> DB columns
FIELD_MAP = {
    "name": "title",
    "description": "description",
   
}

# ---- Add your own static/custom columns here ----
CUSTOM_COLUMNS = {
    "status": "published",        # always insert this string
    "photos" : '["products_images/fcd2610d-e4be-421a-a4ad-f854105f173c.jpg"]',
    "price": 50.55,
    "quantity": 100,
    "shop_id": 1,
}
# -----------------------------------------------


In [39]:
def to_sql_value(x):
    # Convert lists/dicts to JSON strings; leave scalars/None as-is
    if isinstance(x, (list, dict)):
        return json.dumps(x, ensure_ascii=False)
    return x

In [40]:
def normalize_description(val):
    """
    Convert description to a clean string with NO brackets/quotes.
    - list -> "a b c"
    - JSON-stringified list -> parsed then joined
    - string with stray brackets/quotes -> strip them
    - anything else -> stringified & stripped
    Returns '' if nothing meaningful.
    """
    if val is None:
        return ""
    # If already a list, join items with spaces
    if isinstance(val, list):
        return " ".join(str(x).strip() for x in val if str(x).strip()).strip()
    # If string, try to detect and parse '["...","..."]'
    if isinstance(val, str):
        s = val.strip()
        # Try JSON-parse only if it looks like a JSON array
        if s.startswith("[") and s.endswith("]"):
            try:
                arr = json.loads(s)
                if isinstance(arr, list):
                    return " ".join(str(x).strip() for x in arr if str(x).strip()).strip()
            except Exception:
                pass
        # Otherwise, remove leading/trailing brackets/quotes gently
        # e.g. "[text]" -> "text", "\"text\"" -> "text"
        s = re.sub(r'^\s*[\[\"]+|\s*[\]\"]+\s*$', '', s).strip()
        return s
    # Fallback: stringify
    return str(val).strip()

In [41]:

def open_maybe_gzip(path):
    return gzip.open(path, "rt", encoding="utf-8", errors="ignore") if path.endswith(".gz") \
           else open(path, "r", encoding="utf-8", errors="ignore")

UPSERT_KEY = "name"
TARGET_INSERTS = 2000  
# Connect and introspect table to avoid unknown-column errors
cnx = mysql.connector.connect(**DB_CONFIG)
cur = cnx.cursor()
cur.execute(f"DESCRIBE `{TABLE_NAME}`;")
existing_cols = {row[0] for row in cur.fetchall()}

# Filter mappings to columns that actually exist
field_map = {db_col: json_key for db_col, json_key in FIELD_MAP.items() if db_col in existing_cols}
custom_cols = {db_col: val for db_col, val in CUSTOM_COLUMNS.items() if db_col in existing_cols}

missing = set(FIELD_MAP) - set(field_map)
if missing:
    print(f"[info] Skipping non-existent columns in `{TABLE_NAME}`: {sorted(missing)}")
missing_custom = set(CUSTOM_COLUMNS) - set(custom_cols)
if missing_custom:
    print(f"[info] Skipping non-existent custom columns: {sorted(missing_custom)}")

all_cols = list(field_map.keys()) + list(custom_cols.keys())
if not all_cols:
    cur.close(); cnx.close()
    raise RuntimeError("No valid columns to insert after filtering against the table schema.")

col_list = ", ".join(f"`{c}`" for c in all_cols)
placeholders = ", ".join(["%s"] * len(all_cols))

# Build SQL (use upsert if possible)
use_upsert = UPSERT_KEY in existing_cols
if use_upsert:
    update_clause = ", ".join(f"`{c}`=VALUES(`{c}`)" for c in all_cols if c != UPSERT_KEY)
    insert_sql = f"INSERT INTO `{TABLE_NAME}` ({col_list}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE {update_clause};"
else:
    print(f"[info] UPSERT key `{UPSERT_KEY}` not found in `{TABLE_NAME}`. Using plain INSERT.")
    insert_sql = f"INSERT INTO `{TABLE_NAME}` ({col_list}) VALUES ({placeholders});"

inserted = 0
seen = 0
skipped_empty_desc = 0
inserted = 0
processed = 0
skipped_empty_desc = 0
skipped_errors = 0

with open_maybe_gzip(INPUT_FILE) as f:
    for line in f:
        if inserted >= TARGET_INSERTS:
            break
        s = line.strip()
        if not s:
            continue
        try:
            obj = json.loads(s)
        except Exception:
            skipped_errors += 1
            continue

        # --- compute normalized description ONCE and skip row if empty ---
        if desc_db_col:
            desc_key = field_map[desc_db_col]  # the JSON key for description
            desc_norm = normalize_description(obj.get(desc_key))
            if not desc_norm:
                skipped_empty_desc += 1
                processed += 1
                continue

        # Build row values in column order
        row_vals = []
        for db_col, json_key in field_map.items():
            if db_col == desc_db_col:
                row_vals.append(desc_norm)  # use the precomputed normalized description
            else:
                row_vals.append(to_sql_value(obj.get(json_key)))
        row_vals.extend(custom_cols.values())

        try:
            cur.execute(insert_sql, tuple(row_vals))
            inserted += 1
        except mysql.connector.Error as e:
            skipped_errors += 1
        finally:
            processed += 1
            
cnx.commit()
cur.close()
cnx.close()
print(f"Inserted/updated {count} rows into `{TABLE_NAME}`.")


Inserted/updated 500 rows into `products`.
